In [284]:
import pandas as pd

pd.read_csv('/Users/akhil/Downloads/missing_value_new.csv')

,Unnamed: 0,id,Uber Type,Origin Region,Destination Region,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,Journey Distance(m),Departure Date,Departure Time,Travel Time(s),Arrival Time,Fare$
0,0,ID5688329797,2.0,9,4,-38.110916,144.654173,-37.805470,144.997289,46984.0,2018-01-27,12:35:40,11402.76,15:45:42,711.80
1,1,ID3116906741,1.0,1,2,-37.809756,144.963796,-37.811989,144.938314,3041.0,2018-03-05,17:37:55,784.98,17:50:59,10.91
2,2,ID1591370641,0.0,3,4,-37.823615,144.990226,-37.799234,145.001779,3672.0,2018-06-13,05:58:38,912.48,6:13:50,13.17
3,3,ID5734331832,2.0,5,9,-37.805375,144.934730,-38.110916,144.654173,43406.0,2018-04-06,01:53:10,10403.88,4:46:33,653.59
4,4,ID1256422910,0.0,6,2,-37.787442,144.980409,-37.824499,144.952695,5605.0,2018-01-23,04:54:11,1381.44,5:17:12,13.25
5,5,ID3667564621,1.0,3,2,-37.819387,144.983901,-37.822586,144.940219,4377.0,2018-07-12,13:24:23,1026.54,13:41:29,13.49
6,6,ID3994807092,1.0,9,2,-38.110916,144.654173,-37.825523,144.948250,42293.0,2018-02-08,22:27:10,10161.96,1:16:31,50.91
7,7,ID1829814019,0.0,4,7,-37.810811,145.011151,-37.861835,144.905716,12448.0,2018-02-08,09:31:37,3899.16,10:36:36,9.54
8,8,ID3260070053,1.0,9,7,-38.110916,144.654173,-37.861835,144.905716,50797.0,2018-01-09,12:30:43,13204.98,16:10:47,57.30
9,9,ID1287074820,0.0,6,9,-37.773845,144.983689,-38.110916,144.654173,48197.0,2018-02-12,08:12:23,11519.40,11:24:22,21.86


In [285]:
df = pd.read_csv('/Users/akhil/Downloads/missing_value_new.csv')
test = df[df['Fare$'].isna()]
train = df[df['Fare$'].notna()]

In [286]:
from sklearn import preprocessing

def one_hot_encode(df, column_name, prefix):
    new_df = pd.get_dummies(df[column_name],prefix=prefix)
    return pd.concat([df, new_df], axis='columns')

def encode_data(df):
    df.drop(columns =['Unnamed: 0', 'id'], inplace=True)
    fares = df.pop('Fare$')
    
    
    df['Departure Date'] = pd.to_datetime(df['Departure Date'], format='%Y-%m-%d')
    df['Departure Time'] = pd.to_datetime(df['Departure Time'], format='%H:%M:%S')
    df['Arrival Time'] = pd.to_datetime(df['Arrival Time'], format='%H:%M:%S')
    df['Departure Date'] = pd.to_datetime(df['Departure Date']).astype(int)
    df['Departure Time'] = pd.to_datetime(df['Departure Time']).astype(int)
    df['Arrival Time'] = pd.to_datetime(df['Arrival Time']).astype(int)
    
    df = one_hot_encode(df, 'Uber Type', 'uber_type')
    df = one_hot_encode(df, 'Origin Region', 'origin_region')
    df = one_hot_encode(df, 'Destination Region', 'destination_region')
    return df, fares
  
def normalize_data(df):
    np_array = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(np_array)
    np_array_scaled = min_max_scaler.transform(np_array)
    df = pd.DataFrame(np_array_scaled)
    return df, min_max_scaler

def denormalize_data(df, min_max_scaler):
    np_array = df.values
    np_array_scaled = min_max_scaler.inverse_transform(np_array)
    df = pd.DataFrame(np_array_scaled)
    return df

In [288]:
X_train, y_train = encode_data(train.copy())
X_test, y_test = encode_data(test.copy())

X_train.drop(columns =[x for x in X_train.columns if x not in X_test.columns], inplace=True)

X_train, fitter = normalize_data(X_train.copy())
X_test, fitter = normalize_data(X_test.copy())
y_train, min_max_fitter = normalize_data(y_train.copy().to_frame())

In [289]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [290]:
y_pred = regressor.predict(X_test)

In [291]:
y_pred = np.transpose(y_pred)
y_pred


array([[-0.12070089, -0.15636816, -0.18798337, -0.11293215,  0.12264159,
        -0.04869745, -0.12403223, -0.07649262, -0.23798178,  0.27713663,
         0.03741744, -0.06758839, -0.20006755, -0.06921215,  0.45344725,
         0.58355427, -0.03235654, -0.07033746, -0.06663221, -0.19711257]])

In [292]:
y_pred = pd.Series(y_pred[0])
y_pred = denormalize_data(y_pred.copy().to_frame(), min_max_fitter)
y_pred


,0
0,-93.742579
1,-122.717241
2,-148.400168
3,-87.431562
4,103.939119
5,-35.249857
6,-96.448819
7,-57.829543
8,-189.016877
9,229.444712


In [211]:
train

,Unnamed: 0,id,Uber Type,Origin Region,Destination Region,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude,Journey Distance(m),Departure Date,Departure Time,Travel Time(s),Arrival Time,Fare$
0,0,ID5688329797,2.0,9,4,-38.110916,144.654173,-37.805470,144.997289,46984.0,2018-01-27,12:35:40,11402.76,15:45:42,711.80
1,1,ID3116906741,1.0,1,2,-37.809756,144.963796,-37.811989,144.938314,3041.0,2018-03-05,17:37:55,784.98,17:50:59,10.91
2,2,ID1591370641,0.0,3,4,-37.823615,144.990226,-37.799234,145.001779,3672.0,2018-06-13,05:58:38,912.48,6:13:50,13.17
3,3,ID5734331832,2.0,5,9,-37.805375,144.934730,-38.110916,144.654173,43406.0,2018-04-06,01:53:10,10403.88,4:46:33,653.59
4,4,ID1256422910,0.0,6,2,-37.787442,144.980409,-37.824499,144.952695,5605.0,2018-01-23,04:54:11,1381.44,5:17:12,13.25
5,5,ID3667564621,1.0,3,2,-37.819387,144.983901,-37.822586,144.940219,4377.0,2018-07-12,13:24:23,1026.54,13:41:29,13.49
6,6,ID3994807092,1.0,9,2,-38.110916,144.654173,-37.825523,144.948250,42293.0,2018-02-08,22:27:10,10161.96,1:16:31,50.91
7,7,ID1829814019,0.0,4,7,-37.810811,145.011151,-37.861835,144.905716,12448.0,2018-02-08,09:31:37,3899.16,10:36:36,9.54
8,8,ID3260070053,1.0,9,7,-38.110916,144.654173,-37.861835,144.905716,50797.0,2018-01-09,12:30:43,13204.98,16:10:47,57.30
9,9,ID1287074820,0.0,6,9,-37.773845,144.983689,-38.110916,144.654173,48197.0,2018-02-12,08:12:23,11519.40,11:24:22,21.86
